In [19]:
import numpy as np
import pandas as pd

from scipy.stats import f_oneway, mannwhitneyu, shapiro

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score

import time
import warnings

#### Load data

In [50]:
exp = pd.read_csv(r'../../data/clean/gene_expression.csv')
cnv = pd.read_csv(r'../../data/clean/cnv.csv')
cnv_full = pd.read_csv(r'../../data/cnv.csv')
met = pd.read_csv(r'../../data/clean/metabolomics.csv')
labels = pd.read_csv(r'../../data/labels.csv')

In [51]:
exp.drop(columns="Unnamed: 0", inplace=True)
cnv.drop(columns="Unnamed: 0", inplace=True)
cnv_full.drop(columns="Unnamed: 0", inplace=True)
met.drop(columns="Unnamed: 0", inplace=True)
labels.drop(columns="Unnamed: 0", inplace=True)

#### Split labels into classes

In [52]:
emci = labels[labels["DX_bl"] == "EMCI"].reset_index(drop=True)
cn = labels[labels["DX_bl"] == "CN"].reset_index(drop=True)
lmci = labels[labels["DX_bl"] == "LMCI"].reset_index().drop(columns='index')
# ad = labels[labels["DX_bl"] == "AD"].reset_index().drop(columns='index')

targets = pd.concat([cn, emci, lmci]).reset_index(drop=True)
encoding = {"CN": 0, "EMCI": 1, "LMCI": 1}
targets.loc[:, "DX_bl"] = targets["DX_bl"].map(encoding)
targets = targets.astype({"DX_bl": "int16"})

In [55]:
exp = exp.merge(targets[["PTID"]], how='inner', on="PTID")
cnv = cnv.merge(targets[["PTID"]], how='inner', on="PTID")
cnv_full = cnv_full.merge(targets[["PTID"]], how='inner', on="PTID")
met = met.merge(targets[["PTID"]], how='inner', on="PTID")

exp.sort_values(by="PTID", ignore_index=True, inplace=True)
cnv.sort_values(by="PTID", ignore_index=True, inplace=True)
cnv_full.sort_values(by="PTID", ignore_index=True, inplace=True)
met.sort_values(by="PTID", ignore_index=True, inplace=True)
targets.sort_values(by="PTID", ignore_index=True, inplace=True)

#### Get fold 1

In [56]:
n_folds = 5
kf = StratifiedKFold(n_splits=n_folds, random_state=42, shuffle=True)

for i, (train_idx, test_idx) in enumerate(kf.split(np.zeros(targets.shape[0]), targets["DX_bl"])):
    exp_train_1 = exp.loc[train_idx, :]
    cnv_full_train = cnv_full.loc[train_idx, :]
    cnv_full_test = cnv_full.loc[test_idx, :]

    # exp_train = exp.loc[train_idx, :]
    # cnv_train = cnv.loc[train_idx, :]
    # met_train = met.loc[train_idx, :]
    # y_train = targets.loc[train_idx, :]
    # 
    # exp_test = exp.loc[test_idx, :]
    # cnv_test = cnv.loc[test_idx, :]
    # met_test = met.loc[test_idx, :]
    # y_test = targets.loc[test_idx, :]
    break

In [86]:
np.savez(
    r'../../data/testing/fold_1_cnv_full.npz',
    cnv_full_train=cnv_full_train.loc[:, cnv_full_train.columns != "PTID"].to_numpy(),
    cnv_full_test=cnv_full_test.loc[:, cnv_full_test.columns != "PTID"].to_numpy()
)

In [ ]:
np.savez(
    r'../../data/testing/fold_1.npz', 
    exp_train=exp_train.loc[:, exp_train.columns != "PTID"].to_numpy(),
    cnv_train=cnv_train.loc[:, cnv_train.columns != "PTID"].to_numpy(),
    met_train=met_train.loc[:, met_train.columns != "PTID"].to_numpy(),
    exp_test=exp_test.loc[:, exp_test.columns != "PTID"].to_numpy(),
    cnv_test=cnv_test.loc[:, cnv_test.columns != "PTID"].to_numpy(),
    met_test=met_test.loc[:, met_test.columns != "PTID"].to_numpy(),
    y_train=y_train[["DX_bl"]].to_numpy().ravel(),
    y_test=y_test[["DX_bl"]].to_numpy().ravel()
)

#### Load saved fold

In [20]:
npzfile = np.load(r'../../data/testing/fold_1.npz')

exp_train = npzfile["exp_train"]
cnv_train = npzfile["cnv_train"]
met_train = npzfile["met_train"]
y_train = npzfile["y_train"]
exp_test = npzfile["exp_test"]
cnv_test = npzfile["cnv_test"]
met_test = npzfile["met_test"]
y_test = npzfile["y_test"]

In [88]:
npzfile_cnv = np.load(r'../../data/testing/fold_1_cnv_full.npz')

cnv_full_train = npzfile_cnv["cnv_full_train"]
cnv_full_test = npzfile_cnv["cnv_full_test"]

#### Select features

In [96]:
exp_meta = {'pval_thresh': 0.1,  'idx_bnds': (0, exp_train.shape[1] - 1)}
cnv_meta = {'pval_thresh': 0.05, 'idx_bnds': (exp_meta['idx_bnds'][1] + 1, exp_meta['idx_bnds'][1] + cnv_full_train.shape[1])}
met_meta = {'pval_thresh': 0.1,  'idx_bnds': (cnv_meta['idx_bnds'][1] + 1, cnv_meta['idx_bnds'][1] + met_train.shape[1])}

cnv_full_train_log = np.log2(cnv_full_train + 1)
cnv_full_test_log = np.log2(cnv_full_test + 1)

X_train = np.concatenate((exp_train, cnv_full_train_log, met_train), axis=1)
X_test = np.concatenate((exp_test, cnv_full_test_log, met_test), axis=1)

pvals = np.ones((X_train.shape[1]))
for j in range(X_train.shape[1]):
    # feature_data = X_train[:, j].ravel()
    # with warnings.catch_warnings():
        # warnings.simplefilter("ignore")
        # _, p_shapiro = shapiro(feature_data)
        # print(p_shapiro)

    pos = X_train[y_train == 1, j].ravel()
    neg = X_train[y_train == 0, j].ravel()

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # if p_shapiro <= 0.05:
        #     _, p = mannwhitneyu(pos, neg, alternative='two-sided')
        # else:
        _, p = f_oneway(pos, neg)
    pvals[j] = p

pvals = np.nan_to_num(pvals, nan=1.0)
for omic_meta in (exp_meta, cnv_meta, met_meta):
    omic_meta['sig_idx'] = np.where(pvals[omic_meta['idx_bnds'][0]: omic_meta['idx_bnds'][1] + 1] < omic_meta['pval_thresh'])[0]
    print(len(omic_meta['sig_idx']))
print()

# Feature importance based on feature permutation
forest = RandomForestClassifier(random_state=0, class_weight='balanced')
forest.fit(X_train, y_train)
for omic_meta in (exp_meta, cnv_meta, met_meta):
    omic_meta['rf_idx'] = np.where(forest.feature_importances_[omic_meta['idx_bnds'][0]: omic_meta['idx_bnds'][1] + 1] > 0.0)[0]
    print(len(omic_meta['rf_idx']))
print()

# select overlap genes (p<0.1 and importance by RF)
for omic_meta in (exp_meta, cnv_meta, met_meta):
    omic_meta['intxn'] = list(set(omic_meta['sig_idx']) & set(omic_meta['rf_idx']))
    print(len(omic_meta['intxn']))


1285
3330
2

3342
1206
7

308
118
2


#### Run classification

In [97]:
for omic_meta in exp_meta, cnv_meta, met_meta:
    omic_meta['sel_train'] = X_train[:, omic_meta['intxn']]
    omic_meta['sel_test'] = X_test[:, omic_meta['intxn']]
    omic_meta['scaler'] = StandardScaler()

    if len(omic_meta['intxn']) > 0:
        omic_meta['sel_train'] = omic_meta['scaler'].fit_transform(omic_meta['sel_train'])
        omic_meta['sel_test'] = omic_meta['scaler'].transform(omic_meta['sel_test'])

X_train = np.concatenate((exp_meta['sel_train'], cnv_meta['sel_train'], met_meta['sel_train']), axis=1)
X_test = np.concatenate((exp_meta['sel_test'], cnv_meta['sel_test'], met_meta['sel_test']), axis=1)

In [98]:
# tuning RF prediction parameters by exhaust grid search to optimize a prediction RF model
start_time = time.time()
param_grid = {
    'n_estimators': np.arange(40, 260, 20),
    'max_features': np.arange(0.2, 1, 0.1),
    'max_depth': np.arange(1, 10, 1),
    'max_samples': np.arange(0.2, 1, 0.1)
}
            
model_grid = GridSearchCV(
    RandomForestClassifier(criterion='gini', random_state=0, class_weight="balanced"),
    param_grid, 
    n_jobs=-1
).fit(X_train, y_train)

model = model_grid.best_estimator_
elapsed_time = time.time() - start_time


print(f"Elapsed time to tune RF: {elapsed_time:.3f} seconds")
print(f"Best parameters: {model.get_params()}")

# predict using the model built
test_prob = model.predict_proba(X_test)
test_predict = model.predict(X_test)
AUC = roc_auc_score(y_test, test_prob[:, 1])
ACC = accuracy_score(y_test, test_predict)
print("Test accuracy: %f" % ACC)
print("Test      AUC: %f" % AUC)

# prepare the result output
test_out = pd.DataFrame(test_prob)
test_out['predict'] = test_predict
test_out['class_label'] = y_test

Elapsed time to tune RF: 3794.642 seconds
Best parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'max_features': 0.5000000000000001, 'max_leaf_nodes': None, 'max_samples': 0.30000000000000004, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 220, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}
Test accuracy: 0.614754
Test      AUC: 0.491775


In [13]:
test_out.to_csv(r'../../results/multiomics_emci-cn_rf_pval-0.1-0.05-0.1_fold-1.csv')